In [1]:
import numpy as np
import math
import os
import time
import logging
from hyperopt.plotting import *
from hyperopt import fmin, tpe, hp, STATUS_OK, STATUS_FAIL, Trials, base, rand
from data.data_loader import Dataset
from misc.preferences import PREFERENCES
from misc.run_configuration import from_hyperopt, OutputLayerType, LearningSchedulerType, OptimizerType, default_params
from misc import utils
from misc.hyperopt_space import *

from optimizer import get_optimizer
from criterion import NllLoss, LossCombiner
from models.transformer.encoder import TransformerEncoder
from models.jointAspectTagger import JointAspectTagger
from trainer.train import Trainer
import pprint
import argparse
import pickle
import traceback


/Users/felix/Documents/Repositories/TUM/ABSA-Transformer/misc/run_configuration.py:414: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(config.model_size % config.n_heads == 0, f'number of heads {config.n_heads} is not a valid number of heads for model size {config.model_size}.')


In [2]:
POSSIBLE_DATASET_VALUES = ['germeval', 'organic', 'coNLL-2003', 'amazon']


def load_model(dataset, rc, experiment_name):
    transformer = TransformerEncoder(dataset.source_embedding,
                                        hyperparameters=rc)

    # NER or ABSA-task?
    if rc.task == 'ner':
        from models.transformer_tagger import TransformerTagger
        from models.output_layers import SoftmaxOutputLayer
        loss = NllLoss(dataset.target_size, dataset.class_weights[0])
        softmax = SoftmaxOutputLayer(rc.model_size, dataset.target_size)
        model = TransformerTagger(transformer, softmax)

    else:
        from models.jointAspectTagger import JointAspectTagger
        loss = LossCombiner(4, dataset.class_weights, NllLoss)
        model = JointAspectTagger(transformer, rc, 4, 20, dataset.target_names)


    optimizer = get_optimizer(model, rc)
    trainer = Trainer(
                        model,
                        loss,
                        optimizer,
                        rc,
                        dataset,
                        experiment_name,
                        enable_tensorboard=False,
                        verbose=False)
    return trainer

def load_dataset(rc, logger, task):
    dataset = Dataset(
            task,
            logger,
            rc,
            source_index=PREFERENCES.source_index,
            target_vocab_index=PREFERENCES.target_vocab_index,
            data_path=PREFERENCES.data_root,
            train_file=PREFERENCES.data_train,
            valid_file=PREFERENCES.data_validation,
            test_file=PREFERENCES.data_test,
            file_format=PREFERENCES.file_format,
            init_token=None,
            eos_token=None
        )
    dataset.load_data(dsl, verbose=False)
    return dataset

def objective(parameters):
    run_time = time.time()

    utils.reset_loggers()
    experiment_name = utils.create_loggers(experiment_name=main_experiment_name)
    logger = logging.getLogger(__name__)
    dataset_logger = logging.getLogger('data_loader')

    # generate hp's from parameters
    try:
        rc = from_hyperopt(parameters, use_cuda, model_size=300, early_stopping=5, num_epochs=35, log_every_xth_iteration=-1, language=PREFERENCES.language)
    except Exception as err:
        print('Could not convert params: ' + str(err))
        logger.exception("Could not load parameters from hyperopt configuration: " + parameters)
        return {
            'status': STATUS_FAIL,
            'eval_time': time.time() - run_time
        }
    logger.info('New Params:')
    logger.info(rc)
    print('\n\n#########################################################################')
    print(rc)

    logger.debug('Load dataset')
    try:
        dataset = load_dataset(rc, dataset_logger, rc.task)
    except Exception as err:
        print('Could not load dataset: ' + str(err))
        logger.exception("Could not load dataset")
        return {
            'status': STATUS_FAIL,
            'eval_time': time.time() - run_time
        }
    logger.debug('dataset loaded')
    logger.debug('Load model')

    try:
        trainer = load_model(dataset, rc, experiment_name)
    except Exception as err:
        print('Could not load model: ' + str(err))
        logger.exception("Could not load model")
        return {
            'status': STATUS_FAIL,
            'eval_time': time.time() - run_time
        }

    logger.debug('model loaded')

    logger.debug('Begin training')
    model = None
    try:
        result = trainer.train(use_cuda=rc.use_cuda, perform_evaluation=False)
        model = result['model']
    except Exception as err:
        print('Exception while training: ' + str(err))
        logger.exception("Could not complete iteration")
        return {
            'status': STATUS_FAIL,
            'eval_time': time.time() - run_time,
            'best_loss': trainer.get_best_loss(),
            'best_f1': trainer.get_best_f1()
        }

    if math.isnan(trainer.get_best_loss()):
        print('Loss is nan')
        return {
            'status': STATUS_FAIL,
            'eval_time': time.time() - run_time,
            'best_loss': trainer.get_best_loss(),
            'best_f1': trainer.get_best_f1()
        }

    # perform evaluation and log results
    result = None
    try:
        result = trainer.perform_final_evaluation(use_test_set=True, verbose=False)
    except Exception as err:
        logger.exception("Could not complete iteration evaluation.")
        print('Could not complete iteration evaluation: ' + str(err))
        return {
            'status': STATUS_FAIL,
            'eval_time': time.time() - run_time,
            'best_loss': trainer.get_best_loss(),
            'best_f1': trainer.get_best_f1()
        }
    print(f'VAL f1\t{trainer.get_best_f1()} - ({result[1][1]})')
    print(f'VAL loss\t{trainer.get_best_loss()}')

    print(f"       .---.\n \
         /     \\\n\
          \\.@-@./\n\
          /`\\_/`\\\n\
         //  _  \\\\\tLoss: {trainer.get_best_loss()}\n\
        | \\     )|_\tf1: {trainer.get_best_f1()}\n\
       /`\\_`>  <_/ \\\n\
       \\__/'---'\\__/\n")

    return {
            'loss': result[1][0],
            'status': STATUS_OK,
            'eval_time': time.time() - run_time,
            'best_loss': trainer.get_best_loss(),
            'best_f1': trainer.get_best_f1(),
            'sample_iterations': trainer.get_num_samples_seen(),
            'iterations': trainer.get_num_iterations(),
            'rc': rc,
            'results': {
                'train': {
                    'loss': result[0][0],
                    'f1': result[0][1]
                },
                'validation': {
                    'loss': result[1][0],
                    'f1': result[1][1]
                },
                'test': {
                    'loss': result[2][0],
                    'f1': result[2][1]
                }
            }
        }

In [3]:
dataset_choice = 'coNLL-2003'
runs = 100
main_experiment_name = 'CoNLL-2003HyperoptTPE'
use_cuda = True
description = 'CoNLL-2003 NER TPE Hyperopt run'

if dataset_choice not in POSSIBLE_DATASET_VALUES:
    raise Error()

In [9]:
if dataset_choice == POSSIBLE_DATASET_VALUES[0]:
    PREFERENCES.defaults(
        data_root='./data/data/germeval2017',
        data_train='train_v1.4.tsv',    
        data_validation='dev_v1.4.tsv',
        data_test='test_TIMESTAMP1.tsv',
        source_index=0,
        target_vocab_index=2,
        file_format='csv',
        language='de'
    )
    from data.germeval2017 import germeval2017_dataset as dsl

    search_space = {
        'batch_size': hp.quniform('batch_size', 10, 100, 1),
        'num_encoder_blocks': hp.quniform('num_encoder_blocks', 1, 8, 1),
        'pointwise_layer_size': hp.quniform('pointwise_layer_size', 32, 256, 1),
        'clip_comments_to': hp.quniform('clip_comments_to', 10, 250, 1),
        'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.8),
        'output_dropout_rate': hp.uniform('last_layer_dropout', 0.0, 0.8),
        'num_heads': hp.choice('num_heads', [1, 2, 3, 4, 5]),
        'transformer_use_bias': hp_bool('transformer_use_bias'),
        'output_layer': hp.choice('output_layer', [
            {
                'type': OutputLayerType.Convolutions,
                'output_conv_num_filters': hp.quniform('output_conv_num_filters', 1, 400, 1),
                'output_conv_kernel_size': hp.quniform('output_conv_kernel_size', 1, 10, 1),
                'output_conv_stride': hp.quniform('output_conv_stride', 1, 10, 1),
                'output_conv_padding': hp.quniform('output_conv_padding', 0, 5, 1),
            },
            {
                'type': OutputLayerType.LinearSum
            }
        ]),
        'learning_rate_scheduler': hp.choice('learning_rate_scheduler', [
            {
                'type': LearningSchedulerType.Noam,
                'noam_learning_rate_warmup': hp.quniform('noam_learning_rate_warmup', 1000, 9000, 1),
                'noam_learning_rate_factor': hp.uniform('noam_learning_rate_factor', 0.01, 4)
            }
        ]),
        'optimizer': hp.choice('optimizer', [
            {
                'type': OptimizerType.Adam,
                'adam_beta1': hp.uniform('adam_beta1', 0.7, 0.999),
                'adam_beta2': hp.uniform('adam_beta2', 0.7, 0.999),
                'adam_eps': hp.loguniform('adam_eps', np.log(1e-10), np.log(1)),
                'learning_rate': hp.lognormal('adam_learning_rate', np.log(0.01), np.log(10)),
                'adam_weight_decay': 1*10**hp.quniform('adam_weight_decay', -8, -3, 1)
            },
            #{
            #    'type': OptimizerType.SGD,
            #    'sgd_momentum': hp.uniform('sgd_momentum', 0.4, 1),
            #    'sgd_weight_decay': hp.loguniform('sgd_weight_decay', np.log(1e-4), np.log(1)),
            #    'sgd_nesterov': hp_bool('sgd_nesterov'),
            #    'learning_rate': hp.lognormal('sgd_learning_rate', np.log(0.01), np.log(10))
        ]),
        'replace_url_tokens': hp_bool('replace_url_tokens'),
        'harmonize_bahn': hp_bool('harmonize_bahn'),
        'embedding_type': hp.choice('embedding_type', ['fasttext', 'glove']),
        'embedding_name': hp.choice('embedding_name', ['6B']),
        'embedding_dim': hp.choice('embedding_dim', [300]),
        'use_stop_words': hp_bool('use_stop_words'),
        'use_spell_checker': hp_bool('use_spell_checker'),
        'embedding_type': hp.choice('embedding_type', ['fasttext', 'glove']),
        'task': 'germeval'
    }

elif dataset_choice == POSSIBLE_DATASET_VALUES[1]:
     from data.organic2019 import organic_dataset as dsl
     from data.organic2019 import ORGANIC_TASK_ALL, ORGANIC_TASK_ENTITIES, ORGANIC_TASK_ATTRIBUTES, ORGANIC_TASK_ENTITIES_COMBINE, ORGANIC_TASK_COARSE
     PREFERENCES.defaults(
        data_root='./data/data/organic2019',
        data_train='train.csv',    
        data_validation='validation.csv',
        data_test='test.csv',
        source_index=0,
        target_vocab_index=1,
        file_format='csv',
        language='en'
     )

     search_space = {
        'batch_size': hp.quniform('batch_size', 10, 64, 1),
        'num_encoder_blocks': hp.quniform('num_encoder_blocks', 1, 4, 1),
        'pointwise_layer_size': hp.quniform('pointwise_layer_size', 32, 350, 1),
        'clip_comments_to': hp.quniform('clip_comments_to', 45, 180, 1),
        'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.8),
        'output_dropout_rate': hp.uniform('last_layer_dropout', 0.0, 0.8),
        'num_heads': hp.choice('num_heads', [1, 2, 3, 4, 5]),
        'transformer_use_bias': hp_bool('transformer_use_bias'),
        'output_layer': hp.choice('output_layer', [
            {
                'type': OutputLayerType.Convolutions,
                'output_conv_num_filters': hp.quniform('output_conv_num_filters', 10, 400, 1),
                'output_conv_kernel_size': hp.quniform('output_conv_kernel_size', 1, 10, 1),
                'output_conv_stride': hp.quniform('output_conv_stride', 1, 10, 1),
                'output_conv_padding': hp.quniform('output_conv_padding', 0, 5, 1),
            },
            {
                'type': OutputLayerType.LinearSum
            }
        ]),
        'learning_rate_scheduler': hp.choice('learning_rate_scheduler', [
            {
                'type': LearningSchedulerType.Noam,
                'noam_learning_rate_warmup': hp.quniform('noam_learning_rate_warmup', 1000, 9000, 1),
                'noam_learning_rate_factor': hp.uniform('noam_learning_rate_factor', 0.01, 4)
            }
        ]),
        'optimizer': hp.choice('optimizer', [
            {
                'type': OptimizerType.Adam,
                'adam_beta1': hp.uniform('adam_beta1', 0.7, 0.999),
                'adam_beta2': hp.uniform('adam_beta2', 0.7, 0.999),
                'adam_eps': hp.loguniform('adam_eps', np.log(1e-10), np.log(1)),
                'learning_rate': hp.lognormal('adam_learning_rate', np.log(0.01), np.log(10)),
                'adam_weight_decay': 1*10**hp.quniform('adam_weight_decay', -8, -3, 1)
            },
            #{
            #    'type': OptimizerType.SGD,
            #    'sgd_momentum': hp.uniform('sgd_momentum', 0.4, 1),
            #    'sgd_weight_decay': hp.loguniform('sgd_weight_decay', np.log(1e-4), np.log(1)),
            #    'sgd_nesterov': hp_bool('sgd_nesterov'),
            #    'learning_rate': hp.lognormal('sgd_learning_rate', np.log(0.01), np.log(10))
        ]),
        'task': hp.choice('task', [
            ORGANIC_TASK_ENTITIES,
            ORGANIC_TASK_ENTITIES_COMBINE
        ]),
        'use_stop_words': hp_bool('use_stop_words'),
        'use_spell_checker': hp_bool('use_spell_checker'),
        'embedding_type': hp.choice('embedding_type', ['fasttext', 'glove'])
    }
elif dataset_choice == POSSIBLE_DATASET_VALUES[2]:
    PREFERENCES.defaults(
        data_root='./data/data/conll2003',
        data_train='eng.train.txt',
        data_validation='eng.testa.txt',
        data_test='eng.testb.txt',
        source_index=0,
        target_vocab_index=1,
        file_format='txt',
        language='en'
    )
    from data.conll import conll2003_dataset as dsl

    search_space = {
        'batch_size': hp.quniform('batch_size', 10, 64, 1),
        'num_encoder_blocks': hp.quniform('num_encoder_blocks', 1, 4, 1),
        'pointwise_layer_size': hp.quniform('pointwise_layer_size', 32, 350, 1),
        'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.8),
        'output_dropout_rate': hp.uniform('last_layer_dropout', 0.0, 0.8),
        'num_heads': hp.choice('num_heads', [1, 2, 3]),
        'transformer_use_bias': True,
        'learning_rate_scheduler': {
                'type': LearningSchedulerType.Noam,
                'noam_learning_rate_warmup': hp.quniform('noam_learning_rate_warmup', 1000, 9000, 1),
                'noam_learning_rate_factor': hp.uniform('noam_learning_rate_factor', 0.01, 4)
            },
        'optimizer':
            {
                'type': OptimizerType.Adam,
                'adam_beta1': 0.9,
                'adam_beta2': 0.98,
                'adam_eps': 10e-9,
                'learning_rate': hp.lognormal('adam_learning_rate', np.log(0.01), np.log(10)),
                'adam_weight_decay': 1*10**hp.quniform('adam_weight_decay', -8, -3, 1)
            },
        'output_layer': {'type': None},
        'task': 'ner',
        'embedding_type': hp.choice('embedding_type', ['fasttext', 'glove'])
    }
    
else:
    PREFERENCES.defaults(
        data_root='./data/data/amazon/splits',
        data_train='train.pkl',    
        data_validation='val.pkl',
        data_test='test.pkl',
        source_index=0,
        target_vocab_index=1,
        file_format='pkl',
        language='en'
    )
    from data.amazon import amazon_dataset as dsl

In [10]:
experiment_name = utils.create_loggers(experiment_name=main_experiment_name)
logger = logging.getLogger(__name__)
dataset_logger = logging.getLogger('data_loader')
logger.info('Run hyper parameter random grid search for experiment with name ' + main_experiment_name)
logger.info('num_optim_iterations: ' + str(runs))

Log path is  /Users/felix/Documents/Repositories/TUM/ABSA-Transformer/logs/CoNLL-2003HyperoptTPE/20190428/8


In [11]:
try:
    logger.info('Current commit: ' + utils.get_current_git_commit())
    print('Current commit: ' + utils.get_current_git_commit())
except Exception as err:
    logger.exception('Could not print current commit')

trials = Trials()
try:

    best = fmin(objective,
        space=search_space,
        algo=rand.suggest,
        max_evals=runs,
        trials=trials)

    print(best)
except Exception as err:
    logger.exception('Could not complete optimization')
    print('Could not complete optimization. The log file provides more details.')
    traceback.print_tb(err.__traceback__)



path = os.path.join(os.getcwd(), 'logs', f'hp_run_{main_experiment_name}.pkl')
with open(path, 'wb') as f:
    pickle.dump(trials, f)

Current commit: b'5f97528'
Log path is                                          
/Users/felix/Documents/Repositories/TUM/ABSA-Transformer/logs/CoNLL-2003HyperoptTPE/20190428/9
                                                     

#########################################################################
+----------------------------------------------------------------------------------+
|                                 Hyperparameters                                  |
+------------------------------+---------------------------------------------------+
|          Parameter           |                       Value                       |
+------------------------------+---------------------------------------------------+
|            kwargs            | {'batch_size': 48.0, 'learning_rate_sche[...]rue} |
|          model_size          |                        300                        |
|        early_stopping        |                         5                         |
|           use

unk stevanin                                         
unk +49                                              
unk +6221                                            
unk +91-22-265                                       
unk 1-106                                            
unk 10=                                              
unk 312-408-8720                                     
unk a-rated                                          
unk aa-                                              
unk chemlon                                          
unk comelf                                           
unk five-run                                         
unk humenne                                          
unk january-june                                     
unk kekkila                                          
unk langmore                                         
unk neths                                            
unk palmans                                          
unk pulpex                  

unk shi-ting                                         
unk skhirat                                          
unk skiing-world                                     
unk spiring                                          
unk szonn                                            
unk t-bill                                           
unk takeishi                                         
unk theybers                                         
unk topolcany                                        
unk undersecretary-general                           
unk vialle                                           
unk vinnytsya                                        
unk w-3                                              
unk w145                                             
unk wabia                                            
unk widness                                          
unk wijffels                                         
unk wisinga                                          
unk year-earlier            

unk 38:34.999                                        
unk 38:35.297                                        
unk 38:42.015                                        
unk 392-6                                            
unk 3:54.87                                          
unk 4-                                               
unk 4-104                                            
unk 4-205                                            
unk 408-8787                                         
unk 439.9                                            
unk 446-9                                            
unk 456.8                                            
unk 4:13.353                                         
unk 4:17.696                                         
unk 4:19.699                                         
unk 5,616.21                                         
unk 5-248                                            
unk 5-81                                             
unk 532-8                   

unk kreir                                            
unk krpaco                                           
unk kurdish-controlled                               
unk laasonen                                         
unk lahssini                                         
unk lancien                                          
unk lausberg                                         
unk lavrentyeva                                      
unk lb-2                                             
unk lb-8                                             
unk leatitia                                         
unk liutauras                                        
unk long-on                                          
unk lovebites                                        
unk m&r                                              
unk machnee                                          
unk malakwen                                         
unk malbak                                           
unk mamatuyev               

...

unk n'senga                                          
unk na=not                                           
unk nablus-based                                     
unk naftemboriki                                     
unk nagoum                                           
unk nalkhera                                         
unk nallbani                                         
unk nandrin                                          
unk narcissitic                                      
unk nations-sponsored                                
unk naumchik                                         
unk nb-10                                            
unk nb-11                                            
unk nb-13                                            
unk nb-16                                            
unk nb-4                                             
unk nb-5                                             
unk nb-6                                             
unk nb-7                    

unk qasri                                            
unk qinglan                                          
unk qotbuddin                                        
unk qrtly                                            
unk quac                                             
unk quasi-democratic                                 
unk quequen                                          
unk rabn.cn                                          
unk race-long                                        
unk rail-for                                         
unk rain-curtailed                                   
unk rajapat                                          
unk ramaunaskaite                                    
unk ramthan                                          
unk ramusch                                          
unk ransart                                          
unk ranyon                                           
unk rarely-traded                                    
unk rastegar                

unk staoueli                                         
unk staouelli                                        
unk stat:exempt-ult                                  
unk stavby                                           
unk steady-higher                                    
unk steel-producing                                  
unk stentex                                          
unk stephnopoulos                                    
unk stering                                          
unk stg-plus                                         
unk stickroth                                        
unk stieglmair                                       
unk still-enslaved                                   
unk stocher                                          
unk stock-swap                                       
unk stock-trade                                      
unk storp                                            
unk straight-run                                     
unk straight-set            

unk x211                                             
unk xhumba                                           
unk xuc                                              
unk y-green                                          
unk yakomas                                          
unk yamakage                                         
unk yamassoum                                        
unk yaqiong                                          
unk yarangga                                         
unk yasuto                                           
unk year*                                            
unk year-half                                        
unk yeard                                            
unk yellow-card                                      
unk yeyen                                            
unk young-hoon                                       
unk younnes                                          
unk yr-yr                                            
unk yuan-dong               

KeyboardInterrupt: 